In [1]:
# read midi and query data
# insert it into dataframe

import os
import pandas as pd

dir_query = "Data query MIR-QBSH"
dir_midi = "Database midi MIR-QBSH"

df_query = []
df_midi = []
rel_dis_query = []
    
# read and insert semitone from query to list of dataframe
for file in os.listdir(dir_query):
    if file.endswith(".csv"):
        file_path = os.path.join(dir_query, file)
        fields = ["semitone"]
        temp_df = pd.read_csv(file_path, usecols=fields)
        df_query.append(temp_df.to_numpy().flatten())

# read and insert note_index or semitone from midi to list of dataframe
for file in os.listdir(dir_midi):
    if file.endswith(".csv"):
        file_path = os.path.join(dir_midi, file)
        fields = ["note_index"]
        temp_df = pd.read_csv(file_path, usecols=fields)
        df_midi.append(temp_df.to_numpy().flatten())

In [2]:
# calculate relative distance
def calc_rel_dis(df):
    res = []
    length = len(df)
    for i in range(length-1):
        dis = float(df[i+1] - df[i])
        res.append(dis)
    res = remove_consecutive(res)
    return res

def remove_consecutive(list):
    i = 0
    while i < len(list)-1:
        if list[i] == list[i+1]:
            del list[i]
        else:
            i = i+1
    return list

dis_query = calc_rel_dis(df_query[0])
dis_midi = calc_rel_dis(df_midi[0])
print(dis_query)
print(dis_midi)

# print(df_query[0])
# print(len(df_query[0]))

# print(df_midi[0])
# print(len(df_midi[0]))

[0.0, 6.0, 2.0, 0.0, -1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, -1.0, 0.0, -1.0, 0.0, -1.0, 0.0, 1.0, 0.0, -1.0, 0.0, -1.0, 0.0, -1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, -1.0, 0.0, -1.0, 0.0, -1.0, 0.0, -1.0, 0.0, -1.0, 0.0, -1.0, 0.0, -1.0, 0.0, -1.0, 0.0, 2.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, -1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, -1.0, 1.0, 0.0, -1.0, 0.0, -1.0, 0.0, -1.0, 0.0, -1.0, 0.0, -1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, -1.0, 0.0, -1.0, 0.0, -1.0, 0.0, -1.0, 0.0, 1.0, 0.0, -1.0, 0.0, 1.0, 0.0]
[2.0, 0.0, -2.0, 2.0, -4.0, -5.0, 5.0, 2.0, 0.0, -2.0, 2.0, -4.0, 0.0, 4.0, 3.0, 0.0, 2.0, -2.0, -1.0, -2.0, 7.0, 0.0, -12.0, 0.0, 12.0, 0.0, -12.0, 5.0, 2.0, 0.0, -2.0, 2.0, -4.0, -5.0, 5.0, 2.0, 0.0, -2.0, 2.0, -4.0]


In [4]:
import edit_distance

ref = dis_midi
hyp = dis_query

sm = edit_distance.SequenceMatcher(a=ref, b=hyp)
print(sm.get_opcodes())
print(sm.ratio())
print(sm.get_matching_blocks())

[['delete', 0, 1, 0, 0], ['delete', 1, 2, 0, 0], ['equal', 2, 3, 0, 1], ['delete', 3, 4, 0, 0], ['delete', 4, 5, 0, 0], ['delete', 5, 6, 0, 0], ['delete', 6, 7, 0, 0], ['delete', 7, 8, 0, 0], ['delete', 8, 9, 0, 0], ['delete', 9, 10, 0, 0], ['delete', 10, 11, 0, 0], ['delete', 11, 12, 0, 0], ['delete', 12, 13, 0, 0], ['delete', 13, 14, 0, 0], ['delete', 14, 15, 0, 0], ['delete', 15, 16, 0, 0], ['delete', 16, 17, 0, 0], ['delete', 17, 18, 0, 0], ['delete', 18, 19, 0, 0], ['delete', 19, 20, 0, 0], ['delete', 20, 21, 0, 0], ['delete', 21, 22, 0, 0], ['delete', 22, 23, 0, 0], ['delete', 23, 24, 0, 0], ['delete', 24, 25, 0, 0], ['delete', 25, 26, 0, 0], ['delete', 26, 27, 0, 0], ['delete', 27, 28, 0, 0], ['delete', 28, 29, 0, 0], ['delete', 29, 30, 0, 0], ['delete', 30, 31, 0, 0], ['delete', 31, 32, 0, 0], ['delete', 32, 33, 0, 0], ['delete', 33, 34, 0, 0], ['delete', 34, 35, 0, 0], ['delete', 35, 36, 0, 0], ['delete', 36, 37, 0, 0], ['delete', 37, 38, 0, 0], ['delete', 38, 39, 0, 0], ['del